# Session 5. 데이터수집과 관리
* 실습코드를 활용해서, 최근 일주일 간의 일일 박스오피스 50위까지의 데이터를 selenium 과 beautiful soup를 활용해서 가져오는 코드를 작성해주세요 ! 
* 가져온 데이터를 날짜별로 dataframe을 만든 후, sqlite3를 이용해 dailyboxoffice.db 형태로 저장해주세요.
* db 안에는 날짜별로 나눠진 총 7개의 테이블이 포함되어 있어야 합니다. 
* 작성한 코드와 db 파일을 올려주시면 됩니다

In [97]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
import time
import requests
from selenium.webdriver.common.keys import Keys
import datetime
import numpy as np

# set which band of date to cover
start = datetime.datetime(2019,3,20) # start date
n = 8 # number of days
end = start + datetime.timedelta(days = n)

### Run Webdriver
- set webdriver to be headless
- click the buttons 4 times each
- get the html source and terminate the driver

In [4]:
# set the driver
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
path = '../../chromedriver'
driver = webdriver.Chrome(path)

# get the daily box office from start date to end date
start_str = start.strftime('%Y-%m-%d')
end_str = 
url = "http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search \ 
    &sSearchFrom={}&sSearchTo={}&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=".format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
driver.get(url)
driver.implicitly_wait(3)

In [21]:
# click 4 times for each 'load more' button
btn = driver.find_elements_by_xpath("//button[@title='더보기']")
btn
for b in btn:
    for _ in range(4):
        try: 
            b.click()
            driver.implicitly_wait(3)
        except: pass

In [57]:
# get html and quit driver
content = driver.find_element_by_xpath("//section[@id='content']").get_attribute('innerHTML')
content = bs(content, 'html.parser')
driver.quit()


<span class="contentBg"></span>
<!-- Path -->
<div class="title_area">
<h3><img alt="일별" src="/kobis/web/comm/images/stat/title_dayBoxes.gif"/></h3>
<div class="path"><a href="/kobis/business/main/main.do"><img alt="HOME" src="/kobis/web/comm/images/common/ico_home.gif"/></a>
<img alt="&gt;" src="/kobis/web/comm/images/common/bu_path.gif"/>
<a href="/kobis/business/stat/boxs/findDailyBoxOfficeList.do">박스오피스</a>
<img alt="&gt;" src="/kobis/web/comm/images/common/bu_path.gif"/>
<a href="/kobis/business/stat/boxs/findDailyBoxOfficeList.do">박스오피스</a>
<img alt="&gt;" src="/kobis/web/comm/images/common/bu_path.gif"/> <strong>일별</strong>
</div> </div>
<!-- //Path -->
<!-- 페이지설명 -->
<div class="explanation_area04">
<ul>
<li><b>[박스오피스]</b>코너는 실시간 발권데이터를 전일기준까지 반영하여 일별/주간/주말/기간별 등 각종 통계정보를 제공합니다.</li>
<li>매일 24시 이후 전환/제공되는 [전일자 통계정보]는 상영마감 및 보정처리 등의 사유로 익일 오전까지 계속 업데이트 되며, <font style="text-decoration: underline;">일마감 후 데이터보정 등의 사유로 통계정보는 변동 될 수 있음을 참고하시기 바랍니다.</font></li>
<li>통계이용안내</li>
<li c

### Get Contents of table
section#content > div.listArea >> table.boardList03 > tbody >> tr >> td

In [74]:
# get the list of tables each representing specific date
div = content.find_all('div', {'class':'listArea'})[:n]
table = [d.find('table',{'class':'boardList03'}).tbody for d in div]

8

In [93]:
data = dict() # dictionary with keys to be date and values to be matrix

In [112]:
for i in range(n): # for n number of days
    t = table[i]
    day = (start + datetime.timedelta(days = i)).strftime('%Y%m%d')
    matrix = np.vstack([[td.text.strip().split('\n')[0] for td in tr.find_all('td')] for tr in t.find_all('tr')])
    df = pd.DataFrame(matrix, columns=['rank', 'title', 'date_released', 'revenue', 'revenue_sh', 'revenue_var',
                                    'revenue_cum', 'view', 'view_var', 'view_cum', 'screen', 'screen_time'])
    data[day] = df

### Save into Database

In [118]:
import sqlite3

con = sqlite3.connect("output/daily_boxoffice.db")
for k in data:
    data[k].to_sql("boxoffice_{}".format(k), con, chunksize=1000, if_exists='replace')
con.commit()
con.close()